# 目標：pm2.5 監測小工具

## 目的：

### 製作出一個可以觀測台灣各地空氣品質的小工具


## 動機：
###  《環境健康透視》期刊，2019年9月25日發布了為期5年的另一則研究，指出短期暴露於高空氣汙染，與兒童精神科就診
### 人數增加有所相關。美國俄亥俄州辛辛納提大學（University of Cincinnati）與辛辛納提兒童醫院追蹤精神病患者居住地
### 區的細懸浮微粒（PM2.5）濃度，研究人員發現，每當PM2.5一升高，接下來幾天就診人數也會隨之變多。污染最高峰時
### 的就診患者，通常與精神分裂症有關，隨後幾天則是出現適應障礙、自殺傾向的病患就診。近年來，台灣的空氣品質也越
### 來越差，因此決定做一個的可以隨時監測台灣pm2.5的小工具


## 遇到的困難：
### 本來想要直接用爬蟲的方式，把資料爬下來，但是這樣的呈現方式有點無趣


## 解決方法：
### 改成用一個小視窗的方式呈現，看起來比較友善

In [ ]:
def rbCity():  #點選縣市選項按鈕後處理函式
    global sitelist, listradio
    sitelist.clear()  #清除原有測站串列
    for r in listradio:  #移除原有測站選項按鈕
        r.destroy()
    n=0
    for c1 in data["County"]:  #逐一取出選取縣市的測站
        if(c1 == city.get()):
            sitelist.append(data.ix[n, 0])
        n += 1    
    sitemake()  #建立測站選項按鈕
    rbSite()  #顯示PM2.5訊息

def rbSite():  #點選測站選項按鈕後處理函式
    n = 0
    for s in data.ix[:, 0]:  #逐一取得測站
        if(s == site.get()):  #取得點選的測站
            pm = data.ix[n, "PM2.5"]  #取得PM2.5的值
            if(pd.isnull(pm)):  #如果沒有資料
                result1.set(s + "站的 PM2.5 值目前無資料！")
            else:  #如果有資料
                if(pm <= 35):  #轉換為等級
                    grade1 = "低"
                elif(pm <= 53):
                    grade1 = "中"
                elif(pm <= 70):
                    grade1 = "高"
                else:
                    grade1 = "非常高"
                result1.set(s + "站的 PM2.5 值為「" + str(pm) + "」：「" + grade1 + "」等級")
            break  #找到點選測站就離開迴圈
        n += 1
    
def clickRefresh():  #重新讀取資料
    global data
    data = pd.read_csv("http://opendata.epa.gov.tw/webapi/Data/REWIQA/?$orderby=SiteName&$skip=0&$top=1000&format=csv")
    rbSite()  #更新測站資料

def sitemake():  #建立測站選項按鈕
    global sitelist, listradio
    for c1 in sitelist:  #逐一建立選項按鈕
        rbtem = tk.Radiobutton(frame2, text=c1, variable=site, value=c1, command=rbSite)  #建立選項按鈕
        listradio.append(rbtem)  #加入選項按鈕串列
        if(c1==sitelist[0]):  #預設選取第1個項目         
            rbtem.select()
        rbtem.pack(side="left")  #靠左排列

import tkinter as tk
import pandas as pd

data = pd.read_csv("http://opendata.epa.gov.tw/webapi/Data/REWIQA/?$orderby=SiteName&$skip=0&$top=1000&format=csv")

win=tk.Tk()
win.geometry("640x270")
win.title("PM2.5 實時監測")

city = tk.StringVar()  #縣市文字變數
site = tk.StringVar()  #測站文字變數
result1 = tk.StringVar()  #訊息文字變數
citylist = []  #縣市串列
sitelist = []  #鄉鎮串列
listradio = []  #鄉鎮選項按鈕串列

#建立縣市串列
for c1 in data["County"]:  
    if(c1 not in citylist):  #如果串列中無該縣市就將其加入
        citylist.append(c1)
#建立第1個縣市的測站串列
count = 0
for c1 in data["County"]:  
    if(c1 ==  citylist[0]):  #是第1個縣市的測站
        sitelist.append(data.ix[count, 0])
    count += 1

label1 = tk.Label(win, text="縣市：", pady=6, fg="blue", font=("新細明體", 12))
label1.pack()
frame1 = tk.Frame(win)  #縣市容器
frame1.pack()
for i in range(0,3):  #3列選項按鈕
    for j in range(0,8):  #每列8個選項按鈕
        n = i * 8 + j  #第n個選項按鈕
        if(n < len(citylist)):
            city1 = citylist[n]  #取得縣市名稱
            rbtem = tk.Radiobutton(frame1, text=city1, variable=city, value=city1, command=rbCity)  #建立選項按鈕
            rbtem.grid(row=i, column=j)  #設定選項按鈕位置
            if(n==0):  #選取第1個縣市
                rbtem.select()

label2 = tk.Label(win, text="測站：", pady=6, fg="blue", font=("新細明體", 12))
label2.pack()
frame2 = tk.Frame(win)  #測站容器
frame2.pack()
sitemake()

btnDown = tk.Button(win, text="更新資料", font=("新細明體", 12), command=clickRefresh)
btnDown.pack(pady=6)
lblResult1 = tk.Label(win, textvariable=result1, fg="red", font=("新細明體", 16))
lblResult1.pack(pady=6)
rbSite()  #顯示測站訊息

win.mainloop()

## 執行以後就會看見小視窗

如果圖片跑不出來可以看 https://drive.google.com/drive/folders/1bRla-t6vhnUbbk8H7xL4eroBL4do3cBa?usp=sharing 

可以根據想要查詢的城市去點選

![pho1](images/pho1.png)